In [10]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import geopandas as gp
from collections import defaultdict

In [3]:
mat_contents = loadmat("../../graph_convolutional_clustering/data/wiki.mat")

In [4]:
mat_contents

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sat Feb 16 19:08:56 2019',
 '__version__': '1.0',
 '__globals__': [],
 'fea': <2405x4973 sparse matrix of type '<class 'numpy.float64'>'
 	with 1556595 stored elements in Compressed Sparse Column format>,
 'gnd': array([[9],
        [9],
        [9],
        ...,
        [9],
        [9],
        [9]], dtype=uint8),
 'W': <2405x2405 sparse matrix of type '<class 'numpy.float64'>'
 	with 24357 stored elements in Compressed Sparse Column format>}

In [7]:
mat_contents["W"].shape
mat_contents["fea"].shape

(2405, 4973)

In [8]:
np.unique(mat_contents["gnd"])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
      dtype=uint8)

In [9]:
def calculate_adjacency(gdf, unique_col):
    '''
    Code that takes a geodataframe and returns two dataframes: the first with rook adjacencies and the second with queen adjacencies.
    
    Both dataframes have two columns the first column with the unique column values, and the second column with a list of adjacent geometries, listed by their unique column value.
    ''' 
    
    # Confirm that unique_col is actually unique
    if not(max(gdf[unique_col].value_counts(dropna = False) == 1)):
        raise ValueError("Non-unique column provided")
    
    # Intersected the GeoDataFrame with the buffer with the original GeoDataFrame
    all_intersections = gp.overlay(gdf, gdf, how = "intersection", keep_geom_type = False)
   
    # Filter out self-intersections
    filtered_intersections = all_intersections[all_intersections[unique_col+"_1"]!=all_intersections[unique_col+"_2"]]
    
    # Separate out point intersections
    point_intersections = filtered_intersections[filtered_intersections.geom_type == "Point"]
    non_point_intersections = filtered_intersections[filtered_intersections.geom_type != "Point"]
    
    # Define a tuple of zips of the unique_col pairs present in the non-point intersections
    non_point_intersections_tuples = tuple(zip(non_point_intersections[unique_col+"_1"], non_point_intersections[unique_col+"_2"]))
    
    # Define a dictionary that will map from a unique_col value to a list of other unique_cols it is adjacent to
    rook_dict = defaultdict(list)
    
    # Iterate over the tuples
    for val in non_point_intersections_tuples:        
        rook_dict[val[0]].append(val[1])

    # Some shapes will only intersect with themselves and not be added to the above
    not_added = list(set(gdf[unique_col]).difference(set(rook_dict.keys())))
    for val in not_added:
        
        # For each of these, add a blank list to the dictionary
        rook_dict[val] = []
     
    # Create DataFrame of rook intersections
    df_rook = pd.DataFrame()
    df_rook['GEOID20'] = rook_dict.keys()
    df_rook["ADJ_GEOMS"] = rook_dict.values()

    return df_rook

In [11]:
df = pd.read_csv("../data/Joined_data.csv")

In [17]:
df

,GEO_ID,NAME_x,DP03_0001E,DP03_0001PE,DP03_0002E,DP03_0002PE,DP03_0003E,DP03_0003PE,DP03_0004E,DP03_0004PE,...,GEOID,NAME_y,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,55001950100,"Census Tract 9501, Adams County, Wisconsin",2631,2631,1249,47.5,1249,47.5,1184,45.0,...,55001950100,9501.00,Census Tract 9501,G5020,S,233951801,17547549,44.202274,-89.746890,"POLYGON ((-89.92702199999999 44.193674, -89.92..."
1,55001950201,"Census Tract 9502.01, Adams County, Wisconsin",1349,1349,765,56.7,765,56.7,663,49.1,...,55001950201,9502.01,Census Tract 9502.01,G5020,S,110142594,7642055,44.003147,-89.887676,"POLYGON ((-90.018767 44.047719, -90.0187009999..."
2,55001950202,"Census Tract 9502.02, Adams County, Wisconsin",2339,2339,1179,50.4,1168,49.9,1056,45.1,...,55001950202,9502.02,Census Tract 9502.02,G5020,S,467891428,50528185,44.088808,-89.770574,POLYGON ((-90.02637799999999 44.08720599999999...
3,55001950400,"Census Tract 9504, Adams County, Wisconsin",3937,3937,2096,53.2,2093,53.2,1901,48.3,...,55001950400,9504.00,Census Tract 9504,G5020,S,235590227,1195071,43.940747,-89.737699,"POLYGON ((-89.937725 43.96629, -89.937377 43.9..."
4,55001950501,"Census Tract 9505.01, Adams County, Wisconsin",1303,1303,676,51.9,676,51.9,614,47.1,...,55001950501,9505.01,Census Tract 9505.01,G5020,S,114019282,19754029,43.881719,-89.896239,"POLYGON ((-89.984191 43.911815, -89.983969 43...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,55141011300,"Census Tract 113, Wood County, Wisconsin",3591,3591,2144,59.7,2144,59.7,2019,56.2,...,55141011300,113.00,Census Tract 113,G5020,S,12013212,1425599,44.364365,-89.837272,"POLYGON ((-89.859371 44.361943, -89.859279 44...."
1388,55141011400,"Census Tract 114, Wood County, Wisconsin",4443,4443,2415,54.4,2415,54.4,2187,49.2,...,55141011400,114.00,Census Tract 114,G5020,S,11657150,341824,44.367309,-89.803386,"POLYGON ((-89.817194 44.386061, -89.817178 44...."
1389,55141011500,"Census Tract 115, Wood County, Wisconsin",4793,4793,3386,70.6,3373,70.4,3208,66.9,...,55141011500,115.00,Census Tract 115,G5020,S,33527893,605288,44.365464,-89.759125,"POLYGON ((-89.80564 44.343726, -89.802447 44.3..."
1390,55141011600,"Census Tract 116, Wood County, Wisconsin",4209,4209,2746,65.2,2746,65.2,2514,59.7,...,55141011600,116.00,Census Tract 116,G5020,S,128689803,4188672,44.284962,-89.798004,"POLYGON ((-89.91256299999999 44.273437, -89.91..."


In [16]:
gp.GeoDataFrame(df,geometry = "geometry")

TypeError: Input must be valid geometry objects: POLYGON ((-89.92702199999999 44.193674, -89.92678599999999 44.194407, -89.926395 44.195107, -89.92631399999999 44.195251, -89.92597599999999 44.195645, -89.92588599999999 44.195844, -89.925608 44.196199, -89.925252 44.196537, -89.924762 44.196892, -89.92462999999999 44.196954, -89.92364599999999 44.19756, -89.92262599999999 44.197897, -89.92070099999999 44.198253, -89.919495 44.19847499999999, -89.91926599999999 44.198474, -89.918464 44.198389, -89.918092 44.19835, -89.916917 44.198287, -89.91548499999999 44.198245, -89.91359399999999 44.198182, -89.913437 44.198184, -89.91164599999999 44.198201, -89.909869 44.198241, -89.90877999999999 44.198549, -89.90840799999999 44.198716, -89.908075 44.198867, -89.90754699999999 44.199104, -89.906773 44.199549, -89.906493 44.199703, -89.90593199999999 44.200215, -89.905371 44.20094599999999, -89.905013 44.201604, -89.904859 44.20226299999999, -89.904888 44.202538, -89.90496 44.203214, -89.9053139999